In [1]:
import random
import pandas as pd
import numpy as np
from scipy import sparse as ss
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import DataStructs
from tqdm import tqdm

## Read the building blocks and create an index for all pairs

In [2]:
df_bbs = pd.read_csv('/projects/ML-SpaceDock/data/CBL-B/building_blocks.tsv', delimiter='\t')
df_bbs

,name,smiles,building_block
0,EN300-6735081_i001,C[C@@H](c1ccc(cc1)Cl)N2C[C@H](CC2=O)C(=O)[O-],Negishi_Halide
1,EN300-6735081_i002,C[C@@H](c1ccc(cc1)Cl)N2C[C@@H](CC2=O)C(=O)[O-],Negishi_Halide
2,EN300-6736861_i001,c1ccc(cc1)/C=C(\c2ccc(cc2)Cl)/C(=O)[O-],Negishi_Halide
3,EN300-151532_i001,c1cc(c(cc1[C@@H](C(=O)[O-])O)F)Cl,Negishi_Halide
4,EN300-151532_i002,c1cc(c(cc1[C@H](C(=O)[O-])O)F)Cl,Negishi_Halide
...,...,...,...
602860,EN300-2773878_i001,c1cc(c(cc1C(=O)[O-])C#N)N(=O)=O,Grignard_Ketone_aldehyde_nitrile
602861,EN300-314391_i001,CC(C)(C)OC(=O)N[C@H](CC#N)C(=O)[O-],Grignard_Ketone_aldehyde_nitrile
602862,EN300-314391_i002,CC(C)(C)OC(=O)N[C@@H](CC#N)C(=O)[O-],Grignard_Ketone_aldehyde_nitrile
602863,EN300-259630_i001,CC(C)(C)OC(=O)N1CC[C@](C1)(C#N)C(=O)[O-],Grignard_Ketone_aldehyde_nitrile


In [3]:
# See number of building blocks by their types
df_bbs.groupby('building_block').size()

building_block
Amide_Amine                            75697
Amide_Carboxylic_acid                  35406
Benzoxazole_Aminophenol                  150
Benzoxazole_Benzaldehyde                4060
Buchwald_Amine                         66153
Buchwald_Arylhalide                    17993
Grignard_Halide                        30243
Grignard_Ketone_aldehyde_nitrile       14532
Negishi_Halide                         28785
Oxadiazole_Carboxylic_acid             40583
Oxadiazole_Nitrile                      6434
Reductive_amination_Amine              34489
Reductive_amination_Ketone_aldehyde    10799
SNAr_Amine                             62973
SNAr_Arylhalide                         3768
Sulfonamide_Amine                      81459
Sulfonamide_Sulfonylchloride            1381
Triazole_Carboxylates_esters           50270
Triazole_Nitrile                        6180
Williamson_ether_Alcohol               22217
Williamson_ether_Halide                 9293
dtype: int64

In [4]:
# Enumerating building block types
for i, (name, group) in enumerate(df_bbs.groupby('building_block')):
    print(i, name)

0 Amide_Amine
1 Amide_Carboxylic_acid
2 Benzoxazole_Aminophenol
3 Benzoxazole_Benzaldehyde
4 Buchwald_Amine
5 Buchwald_Arylhalide
6 Grignard_Halide
7 Grignard_Ketone_aldehyde_nitrile
8 Negishi_Halide
9 Oxadiazole_Carboxylic_acid
10 Oxadiazole_Nitrile
11 Reductive_amination_Amine
12 Reductive_amination_Ketone_aldehyde
13 SNAr_Amine
14 SNAr_Arylhalide
15 Sulfonamide_Amine
16 Sulfonamide_Sulfonylchloride
17 Triazole_Carboxylates_esters
18 Triazole_Nitrile
19 Williamson_ether_Alcohol
20 Williamson_ether_Halide


In [5]:
# Creating reactions rules and dictionaries
# Reaction rules - 2D np.array each row correspond to a reaction, and each element in a row correspont to the building block participating in the reaction
# IMPORTANT: the nuber two the left should be the bb1 and the number to the right should be bb2
reaction_rules = np.array([
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7],
    [8, 8],
    [10, 9],
    [11, 12],
    [13, 14],
    [15, 16],
    [18, 17],
    [19, 20],
])
# Reactions - dictianary, for each name of reaction where is a corresponding number - row number in the reaction_rules array
reactions_names = {
    'Amide': 0, 
    'Benzoxazole': 1,
    'Buchwald': 2,
    'Grignard': 3,
    'Negishi': 4,
    'Oxadiazole': 5,
    'Reductive_amination': 6,
    'SNAr': 7,
    'Sulfonamide': 8,
    'Triazole': 9,
    'Williamson_ether': 10
}
reactions_numbers = {
    0: 'Amide', 
    1: 'Benzoxazole',
    2: 'Buchwald',
    3: 'Grignard',
    4: 'Negishi',
    5: 'Oxadiazole',
    6: 'Reductive_amination',
    7: 'SNAr',
    8: 'Sulfonamide',
    9: 'Triazole',
    10: 'Williamson_ether'
}
#np.save('../data/CBL-B/reactions_rules.npy', reaction_rules)

In [6]:
# np.array of building blocks dataframes
bbs = np.array([group.reset_index(drop=True) for name, group in df_bbs.groupby('building_block')], dtype=object)
# number of pairs for each reaction:
pairs_per_reaction = np.array([len(bbs[reaction[0]])*len(bbs[reaction[1]]) for reaction in reaction_rules])
# indexes of the last pair for each reaction:
reaction_borders = np.cumsum(pairs_per_reaction) - 1

In [7]:
pairs_per_reaction

array([2680127982,     609000, 1190290929,  439491276,  828576225,
        261111022,  372446711,  237282264,  112494879,  310668600,
        206462581])

In [8]:
reaction_borders

array([2680127981, 2680736981, 3871027910, 4310519186, 5139095411,
       5400206433, 5772653144, 6009935408, 6122430287, 6433098887,
       6639561468])

In [9]:
# Overall number of pairs
pairs_per_reaction.sum()

6639561469

In [10]:
# Function, by a given index returnes names and SMILES of the buildingblocks and the number and name of the reaction
def get_pair(idx):
    
    reaction_n = np.searchsorted(reaction_borders, idx)
    
    idx_local = idx - reaction_borders[reaction_n] + pairs_per_reaction[reaction_n] - 1

    bb_1_idx = idx_local // len(bbs[reaction_rules[reaction_n][1]])
    bb_2_idx = idx_local % len(bbs[reaction_rules[reaction_n][1]])

    return [
        bbs[reaction_rules[reaction_n]][0]['name'].iloc[bb_1_idx], 
        bbs[reaction_rules[reaction_n]][0]['smiles'].iloc[bb_1_idx],
        bbs[reaction_rules[reaction_n]][1]['name'].iloc[bb_2_idx],
        bbs[reaction_rules[reaction_n]][1]['smiles'].iloc[bb_2_idx],
        reaction_n,
        reactions_numbers[reaction_n]
        ]

In [11]:
get_pair(4310519191)

['EN300-6735081_i001',
 'C[C@@H](c1ccc(cc1)Cl)N2C[C@H](CC2=O)C(=O)[O-]',
 'EN300-151532_i002',
 'c1cc(c(cc1[C@H](C(=O)[O-])O)F)Cl',
 4,
 'Negishi']

In [12]:
# returns index of a pair by building block names and reaction name
def get_index(bb1, bb2, reaction):
    
    reaction_n = reactions_names[reaction]
    
    bb_1_idx = bbs[reaction_rules[reaction_n][0]].index[bbs[reaction_rules[reaction_n][0]]['name'] == bb1].tolist()[0]
    bb_2_idx = bbs[reaction_rules[reaction_n][1]].index[bbs[reaction_rules[reaction_n][1]]['name'] == bb2].tolist()[0]
    
    idx = reaction_borders[reaction_n] + (bb_1_idx*len(bbs[reaction_rules[reaction_n][1]]) + bb_2_idx) - pairs_per_reaction[reaction_n] + 1
    
    return idx

In [13]:
get_index('EN300-6735081_i001', 'EN300-151532_i002', 'Negishi')

4310519191

## Read hits and get indexes of hits

In [14]:
df_hits = pd.read_csv('../data/CBL-B/hits.tsv', delimiter='\t')
df_hits

,bb1,bb2,reaction,IFP,IFP_polar
0,EN300-1878549_i001,EN300-11626_i001,Amide,0.764706,0.50
1,EN300-3575340_i001,EN300-1589742_i001,Amide,0.647059,0.50
2,EN300-3575340_i002,EN300-7455807_i002,Amide,0.631579,0.50
3,EN300-3575340_i002,EN300-384067_i001,Amide,0.666667,0.50
4,EN300-3575340_i002,EN300-343493_i001,Amide,0.631579,0.50
...,...,...,...,...,...
7702851,EN300-103503_i002,EN300-12411_i001,Williamson_ether,0.611111,0.75
7702852,EN300-103503_i002,EN300-10699143_i001,Williamson_ether,0.611111,0.50
7702853,EN300-103503_i003,EN300-10547_i002,Williamson_ether,0.666667,0.50
7702854,EN300-103503_i003,EN300-7354512_i001,Williamson_ether,0.611111,0.50


In [15]:
def get_hits_idxs(df_hits, q):
    df_hits = df_hits[df_hits['IFP'] >= q]
    hits_idxs = np.array([], dtype=np.int64)
    for name, group in df_hits.groupby('reaction'):
        
        reaction_n = reactions_names[name]
        
        bb1_df = bbs[reaction_rules[reaction_n][0]]
        bb1_df = bb1_df.set_index('name')
        bb2_df = bbs[reaction_rules[reaction_n][1]]
        bb2_df = bb2_df.set_index('name')
        
        bb1_idxs = bb1_df.index.get_indexer(group['bb1'])
        bb2_idxs = bb2_df.index.get_indexer(group['bb2'])
        
        idxs_local = bb1_idxs * len(bb2_df) + bb2_idxs
        idxs = idxs_local + reaction_borders[reaction_n] - pairs_per_reaction[reaction_n] + 1
    
        hits_idxs = np.hstack([hits_idxs, idxs])
    return hits_idxs

In [16]:
for q in [0.6, 0.7, 0.8, 0.9]:
    hits_idxs = get_hits_idxs(df_hits, q)
    #np.save(f'../data/CBL-B/hits_idxs_q_{q}.npy', hits_idxs)

## Generate fingerptints of building blocks

In [17]:
parameters={ 
    "radius": 2,
    "nBits": 2048,
    "invariants": [],
    "fromAtoms": [],
    "useChirality": True,
    "useBondTypes": True,
    "useFeatures": False
}

In [18]:
from rdkit import RDLogger                                                                                                                                                               
RDLogger.DisableLog('rdApp.*')  

In [19]:
for n, df in enumerate(bbs):
    fps = np.zeros((len(df), 2048), np.int8)
    for i, smiles in enumerate(tqdm(df['smiles'], desc="Calculating fingerprints", unit="fp")):
        
        mol = Chem.MolFromSmiles(smiles)
        fp = rdmd.GetMorganFingerprintAsBitVect(mol, **parameters)
        
        fps[i] = np.array(fp).astype(np.int8)

    np.save(f'../data/CBL-B/bb_{n}.npy', fps)

Calculating fingerprints: 100%|███████████| 9293/9293 [00:07<00:00, 1192.37fp/s]
